In [1]:
import warnings
import numpy as np
import pandas as pd
import scipy.optimize as so
from serde.json import from_json, to_json

from data import Donnees

# Extraire les valeurs associées à chaque aliment
Al = pd.read_csv('./Aliments.csv', sep=';', index_col=0)
aliments = np.array(Al).T


def apports(RESULT):
    """La fonction apport permet de calculer les apports de chaque aliments nécessaire pour réaliser l'optimum.

    Args:
        RESULT (_OptimizeResult_): scipy.optimize.OptimizeResult est un objet scipy optimise,
        qui retourne plusieurs éléments en fonction de la réussite de l'optimisation, 
        comme {x} la valeur réalisant l'optimum, {fun} la valeur que prend notre fonction à l'optimum etc.

    Returns:
        REPAS(_pd.dataframe_): dataframe pandas comprenant les quantités 
        de chaque aliments non nul pour réaliser l'optimum, ainsi que le détails
        de ces quantité en valeur our chaque nutriment.
    """
    A = RESULT.x
    u, = A.nonzero()
    REST = Al.iloc[u, :]
    QTE = pd.DataFrame(
        {'Quantités en g': 100*A[A.nonzero()]}).set_index(Al.iloc[u, :].index)
    BILAN = pd.DataFrame(
        (np.array(REST).T*A[A.nonzero()]).T).set_index(QTE.index)
    REPAS = pd.concat([QTE, BILAN], axis=1)
    REPAS = REPAS.append([REPAS.sum()])
    Namescol = ['Quantité (en g)']+list(Al.columns)
    Namescol[-1] = 'Prix (en €)'
    REPAS.columns = Namescol
    Namesrow = list(QTE.index)
    Namesrow.append('Apports du repas')
    REPAS.index = Namesrow
    return REPAS


def rename_aliment(old_name):
    """
    Fonction qui permet d'enlever le type de chaque aliment de sorte à fournir un résultat plus clair.
    """
    new_name = old_name.replace("Lait – ", "")
    new_name = new_name.replace("Viande – ", "")
    new_name = new_name.replace("Poisson – ", "")
    new_name = new_name.replace("Légume – ", "")
    new_name = new_name.replace("Base – ", "")
    return new_name


def resoud(donnees: Donnees) -> str:
    """_summary_

afin de vérifier si l'optimisation est un succès : OptimizeResult.status
    Args:
        donnees (_str_): fichier codé en .json avec les données du problème.

    Returns:
        phrase(_str_): phrase synthétisant le résultat de l'optimisation, en fournissant
        les aliments et les quantités réalisant l'optimum, le prix ainsi que les calorie du repas optimal.
    """
    # Extraire les données de l'objet Donnees
    # nutriments = Donnees.nutriments
    # aliments_index = Donnees.aliments_index
    # contraintes_aliments = Donnees.contraintes_aliments
    # contraintes_couts = Donnees.contraintes_couts
    # besoins_journaliers = Donnees.besoins_journaliers

    # Formater les contraintes des aliments
    coeff = aliments[:-1] 
    bdns = [(0,None) for k in range(41)]
    bdns[38] = (0,5)

    # Formater les contraintes des coûts
    # b_eq = contraintes_couts[0]

    # Formater les besoins journaliers
    # k = int
    with open('../donnees.json', 'r') as f:
        donnees = from_json(Donnees,f.read())
    betaFE = np.array(donnees.betaF)

    # Formater la fonction objectif(minimiser le coût)
    c = aliments[-1]

    # Résoudre le problème d'optimisation linéaire
    
    ## désactiver les avertissements de type "DeprecationWarning"
    warnings.simplefilter('ignore', DeprecationWarning)
    RESULT = so.linprog(c, A_ub=-coeff, b_ub=-betaFE, method='simplex', bounds=bdns)
    if RESULT.success !=True:
        raise ValueError("L'optimisation sans contraintes supplémentaires est un échec")
    RESULTF= so.linprog(c, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaFE,1.1*betaFE),axis=0),method='simplex')
    if RESULTF.success !=True:
        raise ValueError("L'optimisation avec contraintes supplémentaires est un échec")
    
    ## réactiver les avertissements de type "DeprecationWarning"
    warnings.simplefilter('default', DeprecationWarning)
    
    # Formatter la solution sans limite
    A = RESULT.x
    u, = A.nonzero()
    Qt = A[u]
    Phrase = "Un repas sans contrainte supplémentaires est constitué de "
    for s in range(len(u)-1):
        if s > 0:
            Phrase += ","
        gr = Qt[s]*100
        old_name = Al.iloc[u].index[s]
        name = rename_aliment(old_name)
        Phrase += " de {:0.2f} g de {}".format(gr, name)
    s = len(u)-1
    gr = Qt[s]*100
    old_name = Al.iloc[u].index[s]
    name = rename_aliment(old_name)
    Phrase += " et de {:0.2f} g de {}. ".format(gr, name)
    REPAS = apports(RESULT)
    Phrase += " Il coûte un total de {:0.2f} euros et comprend {:0.2f} calories. Nous avons fixé une limite à la quantité de haricots blancs pour avoir un repas plus varié.".format(
        RESULT.fun, REPAS['Energie (kcal)'][-1])
    # Formatter la solution avec limite de 10%
    B = RESULTF.x
    u_2, = B.nonzero()
    Qt = A[u_2]
    Phrase_2 = " Un repas avec 10% de contraintes supplémentaires est constitué de "
    for s in range(len(u_2)-1):
        if s > 0:
            Phrase_2 += ','
        gr = Qt[s]*100
        old_name = Al.iloc[u_2].index[s]
        name = rename_aliment(old_name)
        Phrase_2 += " de {:0.2f} g de {}".format(gr, name)
    s = len(u_2)-1
    gr = Qt[s]*100
    old_name = Al.iloc[u_2].index[s]
    name = rename_aliment(old_name)
    Phrase_2 += " et de {:0.2f} g de {}. ".format(gr, name)
    REPAS = apports(RESULTF)
    Phrase_2 += " Il coûte un total de {:0.2f} euros et comprend {:0.2f} calories.".format(
        RESULTF.fun, REPAS['Energie (kcal)'][-1])
    return Phrase, Phrase_2

In [2]:
import typer
application = typer.Typer()

In [3]:
@application.command()
def calcule(nom_fichier: str):
    with open(nom_fichier, "r") as fichier:
        code = fichier.read()
        
    donnees = from_json(Donnees, code)
    solution = resoud(donnees)
    print(solution)

In [4]:
calcule('../donnees.json')

('Un repas sans contrainte supplémentaires est constitué de  de 42.05 g de Gruyère, de 285.57 g de Frites, de 500.00 g de Haricots blancs, de 7.78 g de Lentilles et de 183.95 g de Semoule.  Il coûte un total de 2.49 euros et comprend 2530.90 calories. Nous avons fixé une limite à la quantité de haricots blancs pour avoir un repas plus varié.', ' Un repas avec 10% de contraintes supplémentaires est constitué de  de 0.00 g de crème fraîche, de 42.05 g de Gruyère, de 0.00 g de Beurre, de 0.00 g de Courge, de 0.00 g de Haricots rouges, de 500.00 g de Haricots blancs et de 183.95 g de Semoule.  Il coûte un total de 5.42 euros et comprend 2000.00 calories.')


C:\Users\karl\AppData\Local\Temp\ipykernel_11400\2077611437.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS = REPAS.append([REPAS.sum()])
C:\Users\karl\AppData\Local\Temp\ipykernel_11400\2077611437.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS = REPAS.append([REPAS.sum()])


In [5]:
with open('../donnees.json', 'r') as f:
        donnees_dict = from_json(Donnees,f.read())

In [6]:
donnees_dict.betaF

[75, 90, 225, 2000, 9, 800, 45]

In [7]:
Al = pd.read_csv('./Aliments.csv', sep=';', index_col=0)
A=np.array(Al).T
coeff=A[:-1] ## matrice des coefficients
z_ori=A[-1]### coefficients de la fonction de coût (prix)
RESULTF=so.linprog(z_ori, A_ub=-coeff, b_ub=-np.array(donnees_dict.betaF),method='simplex') 

C:\Users\karl\AppData\Local\Temp\ipykernel_11400\1011478072.py:5: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTF=so.linprog(z_ori, A_ub=-coeff, b_ub=-np.array(donnees_dict.betaF),method='simplex')


In [8]:
RESULTF

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 2.052827380785162
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
     nit: 20

In [25]:
apports(RESULTF).values[1,1]

C:\Users\karl\AppData\Local\Temp\ipykernel_11400\2077611437.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS = REPAS.append([REPAS.sum()])


74.78538532433176